In [78]:

import numpy as np
from rich.progress import track

class Tanh:
    def activate(self, x):
        return np.tanh(x)
    
    def derivative(self, x):
        return 1 - self.activate(x) ** 2


class Layer:
    def __init__(self, len_inputs, neurons, function, last=False):
        shape = neurons, len_inputs + 1
        self.weights = np.random.uniform(-0.5, 0.5, size=shape)
        self.f = function
        self.last = last
        self.idx = None
        self.neurons = neurons
        self.len_inputs = len_inputs
    
    def forward(self, layer_input):
        self.input = layer_input
        self.net = self.input.dot(self.weights.T)
        self.output = self.f.activate(self.net)
        return self.output
    
    def backward(self, alpha, previous_delta=None, previous_weigth=None, error=None):
        if self.last:
            self.delta = error * self.f.derivative(self.net)
        else:
            self.delta = (np.delete(previous_delta.dot(previous_weigth).T, 0) * self.f.derivative(self.net))
        
        self.weights += np.array([self.delta]).T * np.array([self.input]) * alpha
        
        return self.delta, self.weights
        
        
        
        
    def __repr__(self):
        return f"({self.idx}º Layer, Neurons: {self.neurons}, Last: {self.last})"


class NeuralNetwork:
    def __init__(self, *layers: Layer):
        self.layers = list(layers)
        for idx, layer in enumerate(self.layers):
            layer.idx = idx + 1
        self.layers[-1].last = True
        self.len_inputs = self.layers[0].len_inputs
        
    def __repr__(self):
        return f"NeuralNetwork (Num_Layers: {len(self.layers)}, Len_Inputs: {self.len_inputs}, Layers: {self.layers})"
    
    @property
    def weights(self):
        resp = []
        for idx, layer in enumerate(self.layers):
            resp.append((idx+1, layer.weights))
        return resp
        
    def _forward(self, x_input):
        #input_layer = x_input
        input_layer = np.append(1, x_input)
        for layer in self.layers:
            out_layer = layer.forward(input_layer)
            input_layer = np.append(1, out_layer)
            
        return out_layer
    
    def _backward(self, alpha, error):
        for layer in reversed(self.layers):
            if layer.last:
                previous_delta, previous_weigth = layer.backward(alpha, error=error)
            else:
                previous_delta, previous_weigth = layer.backward(alpha, previous_delta, previous_weigth)
    
    def fit(self, x_train, y_train, epochs=2000, alpha=0.05, batch_size=1):

        for epoch in track(range(epochs), description="Processing..."):
            outputs = []
            batch_errors = []
            data = list(zip(x_train,y_train))
            np.random.shuffle(data)
            x_train,y_train = zip(*data)
            x_train,y_train = np.array(x_train), np.array(y_train)
            for x, y in zip(x_train, y_train):
                out = self._forward(x)
                error = (y - out)

                batch_errors.append(error)
                if len(batch_errors) == batch_size:
                
                    batch_error = sum(batch_errors) / batch_size
                    batch_errors = []
                    self._backward(alpha, batch_error)

                outputs.append(out)
                
            errors = np.array([sum(error) for error in (y_train - outputs) ** 2])
            self.mean_squared_error = sum(errors) / len(errors)
            
            if not epoch % 100:
                print(f"MSE: {self.mean_squared_error}")

                
                
    def predict(self, x):
        out = self._forward(x)
        return out



In [49]:
a = np.array([-1.26103335, -0.61622881])
#b = np.array([-1.10183565, -0.98207112]) 
c = []
c.append(a)
#c.append(b)
sum(c) / 1

array([-1.26103335, -0.61622881])

In [3]:
from rich.traceback import install; install();

In [72]:
x_train = [0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1.0]
y_train = [-.9602, -.5770, -.0729, .3771, .6405, .6600, .4609, .1336, -.2013, -.4344, -.5000 ]
x_train = np.array([[x, x] for x in x_train])
y_train = np.array([[y, y] for y in y_train])


rede = NeuralNetwork(
    Layer(len_inputs=2, neurons=4, function=Tanh()),
    Layer(len_inputs=4, neurons=2, function=Tanh()),
)
rede.fit(x_train, y_train, batch_size=1)

Output()

MSE: 0.9340626169186904

MSE: 0.3815802664894472

MSE: 0.020720467691171047

MSE: 0.008609053311312438

...

MSE: 0.0050644042934896065

MSE: 0.004998638541688824

MSE: 0.004714401944531347

MSE: 0.004544589803795452

MSE: 0.004450287876082404

rede.predict(x_train[145]

In [76]:
import polars as pl

def number_to_neurons(n):
    res = [-1] * 10
    res[n] = 1
    return res

data_train = pl.read_csv("train.csv")

y_train = np.array(data_train.drop_in_place("label"))
y_train = np.array([number_to_neurons(y) for y in y_train])

x_train = np.array([row for row in data_train.rows()]) / 255

In [80]:
rede = NeuralNetwork(
    Layer(len_inputs=784, neurons=28, function=Tanh()),
    Layer(len_inputs=28, neurons=10, function=Tanh()),
)
rede.fit(x_train[:1000], y_train[:1000], epochs=500, batch_size=1)

Output()

MSE: 2.988251082631034

MSE: 0.11612756560550787

MSE: 0.09281698271908441

MSE: 0.0923755768443832

MSE: 0.09227107855480775

In [79]:
rede2 = NeuralNetwork(
    Layer(len_inputs=784, neurons=28, function=Tanh()),
    Layer(len_inputs=28, neurons=10, function=Tanh()),
)
rede2.fit(x_train[:1000], y_train[:1000], epochs=500, batch_size=5)

Output()

MSE: 4.174461249112266

MSE: 0.31335561530602196

MSE: 0.15671465503420604

MSE: 0.12748245455444265

MSE: 0.10666237049060079

In [85]:
def load_weights(rede, n_cam):
    with open("weights.npy", "rb") as f:
        for idx in range(n_cam):
            rede.layers[idx].weights = np.load(f)

load_weights(rede, 2)

In [86]:
rede2 = rede

In [88]:
rede.fit(x_train, y_train, epochs=5000, batch_size=1)

Output()

MSE: 0.1455663463725941

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 rede.fit(x_train, y_train, epochs=5000, batch_size=1)                                        │
│   2                                                                                              │
│                                                                                                  │
│ in fit:97                                                                                        │
│                                                                                                  │
│    94 │   │   │   │   │                                                                          │
│    95 │   │   │   │   │   batch_error = sum(batch_errors) / batch_size                           │
│    96 │   │   │   │   │   batch_errors = []                                                      │
│ ❱  97 │   │   │   │   │   self._backward(alpha, batch_error)                                     │
│    98 │   │   │   │                                                                              │
│    99 │   │   │   │   outputs.append(out)                                                        │
│   100                                                                                            │
│                                                                                                  │
│ in _backward:77                                                                                  │
│                                                                                                  │
│    74 │   │   │   if layer.last:                                                                 │
│    75 │   │   │   │   previous_delta, previous_weigth = layer.backward(alpha, error=error)       │
│    76 │   │   │   else:                                                                          │
│ ❱  77 │   │   │   │   previous_delta, previous_weigth = layer.backward(alpha, previous_delta,    │
│    78 │                                                                                          │
│    79 │   def fit(self, x_train, y_train, epochs=2000, alpha=0.05, batch_size=1):                │
│    80                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [ ]:
rede2.fit(x_train, y_train, epochs=500, batch_size=100)